In [234]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotVisibleException
import pandas as pd
import re
import time 

### Click on Tab

In [232]:
def selectTab(driver, xpath):
    driver.find_element(By.XPATH, xpath).click()
    return driver

### Open up "View More" of Each Race

In [220]:
def viewMore(browser):
    [el.click() for el in browser.find_elements(By.XPATH, "//a[contains(text(),'View More')]")]
    return browser

### Show County Details

In [221]:
def byCounty(browser): 
    for el in browser.find_elements(By.XPATH, "//a[contains(text(),'By County')]"):
        try: 
            el.click()
        except ElementNotVisibleException:
            pass
    return browser
driver = byCounty(driver)

In [222]:
len(driver.find_elements(By.CLASS_NAME, "lead-md_ "))

28

In [224]:
def dataframeRows(browser):
    df_rows = []
    contests = browser.find_elements(By.CLASS_NAME, "contest-wrapper")
    for contest in contests: 
        race_name = contest.find_element(By.CLASS_NAME, "lead-md_").text
        if race_name:
            table = contest.find_element(By.CLASS_NAME, "table")
            headers = [el.text for el in table.find_elements(By.TAG_NAME, "th")]
            rows = table.find_elements(By.TAG_NAME, "tr")
            for row in rows[1:]: 
                split_row = row.text.split("\n")
                if len(split_row) > 1:
                    df_row = [] 
                    county_name = re.sub("[^a-zA-Z]+", "", split_row[0])
                    precincts_reporting = re.sub("[^0-9\s.%]", "", split_row[0])
                    cand_votes = zip(headers[2:], split_row[1:])
                    df_rows.append((race_name, county_name, precincts_reporting, cand_votes))
    return browser, df_rows



In [217]:
final_rows = [[race, county, precincts, cand, votes]for race, county, precincts , cand_votes in df_rows for cand, votes in cand_votes] 
pd.DataFrame(final_rows, columns = ['race name', 'county', 'precincts reporting', 'candidate', 'votes']).to_csv("AZ_statewide.csv", index = False, encoding = 'utf-8')

### Scrape Federal Races

In [242]:
driver = webdriver.Chrome()
driver.get("https://results.arizona.vote/#/federal/4/0")
time.sleep(5) #update to explicit selenium wait
driver = selectTab(driver, "/html/body/div[1]/div/div[1]/div/ul/li[3]/a")
time.sleep(5)
driver = viewMore(driver)
driver = byCounty(driver) 
driver, federal_races_rows = dataframeRows(driver)

### Scrape Statewide Races

In [241]:
driver = selectTab(driver,  "/html/body/div[1]/div/div[1]/div/ul/li[4]/a")
time.sleep(5)
driver = viewMore(driver)
driver = byCounty(driver) 
driver, statewide_races_rows = dataframeRows(driver)

In [244]:
### Combine and create DataFrame

In [245]:
combined = statewide_races_rows + federal_races_rows 
final_rows = [[race, county, precincts, cand, votes]for race, county, precincts , cand_votes in combined  for cand, votes in cand_votes] 

In [246]:

pd.DataFrame(final_rows, columns = ['race name', 'county', 'precincts reporting', 'candidate', 'votes']).to_csv("Arizona.csv", index = False, encoding = 'utf-8')

[(u'U.S. Senator',
  u'Apache',
  u' 100.00%',
  [(u'Green, Angela (GRN)', u'947'),
   (u'McSally, Martha (REP)', u'7,771'),
   (u'Sinema, Kyrsten (DEM)', u'16,148'),
   (u'TOTAL', u'24,866')]),
 (u'U.S. Senator',
  u'Cochise',
  u' 100.00%',
  [(u'Green, Angela (GRN)', u'1,151'),
   (u'McSally, Martha (REP)', u'26,148'),
   (u'Sinema, Kyrsten (DEM)', u'16,736'),
   (u'TOTAL', u'44,035')]),
 (u'U.S. Senator',
  u'Coconino',
  u' 100.00%',
  [(u'Green, Angela (GRN)', u'1,363'),
   (u'McSally, Martha (REP)', u'15,906'),
   (u'Sinema, Kyrsten (DEM)', u'27,874'),
   (u'TOTAL', u'45,143')]),
 (u'U.S. Senator',
  u'Gila',
  u' 100.00%',
  [(u'Green, Angela (GRN)', u'650'),
   (u'McSally, Martha (REP)', u'11,834'),
   (u'Sinema, Kyrsten (DEM)', u'7,396'),
   (u'TOTAL', u'19,880')]),
 (u'U.S. Senator',
  u'Graham',
  u' 100.00%',
  [(u'Green, Angela (GRN)', u'363'),
   (u'McSally, Martha (REP)', u'6,870'),
   (u'Sinema, Kyrsten (DEM)', u'3,368'),
   (u'TOTAL', u'10,601')]),
 (u'U.S. Senator',
